In [4]:
# === Set your fixed electricity price (€/kWh) ===
FIXED_PRICE = 0.1094   # <-- fill in your chosen fixed price

In [5]:
import pandas as pd

# === Configuration ===
FILE_PATH = "jeroen_punt_nl_dynamische_stroomprijzen_jaar_2024.csv"  # your file
CHARGING_HOURS = [18, 19, 20]    # charging from 18:00 to 21:00
POWER_KW = 7.4                   # 1-phase wallbox
DAILY_ENERGY_KWH = 22.2          # total per day (7.4 kW * 3 hours)

# === Load data ===
df = pd.read_csv(FILE_PATH, sep=';', decimal=',')
df['datum_nl'] = pd.to_datetime(df['datum_nl'])
df['hour'] = df['datum_nl'].dt.hour

# === Filter for charging hours ===
df_charge = df[df['hour'].isin(CHARGING_HOURS)].copy()

# === Dynamic price calculation ===
df_charge['cost_dynamic'] = df_charge['prijs_excl_belastingen'] * POWER_KW

# === Fixed price calculation ===
df_charge['cost_fixed'] = FIXED_PRICE * POWER_KW

# === Daily totals ===
daily_costs = (
    df_charge
    .groupby(df_charge['datum_nl'].dt.date)[['cost_dynamic', 'cost_fixed']]
    .sum()
)

# === Year totals ===
total_dynamic = daily_costs['cost_dynamic'].sum()
total_fixed = daily_costs['cost_fixed'].sum()

# === Results ===
print(f"Chosen fixed price: €{FIXED_PRICE:.3f}/kWh")
print(f"Average daily cost (dynamic): €{daily_costs['cost_dynamic'].mean():.2f}")
print(f"Average daily cost (fixed):   €{daily_costs['cost_fixed'].mean():.2f}")
print(f"Total yearly cost (dynamic): €{total_dynamic:.2f}")
print(f"Total yearly cost (fixed):   €{total_fixed:.2f}")
print(f"Yearly difference:           €{total_dynamic - total_fixed:.2f}")


Chosen fixed price: €0.109/kWh
Average daily cost (dynamic): €2.53
Average daily cost (fixed):   €2.43
Total yearly cost (dynamic): €924.88
Total yearly cost (fixed):   €888.90
Yearly difference:           €35.98
